In [2]:
import torch

In [9]:

# ---------- First checkpoint (reference) ----------
sd_ckpt = "/home/deanhazineh/ssd4tb_mounted/cobl/cobl/LDM/SD2p1/v2-1_512-ema-pruned.ckpt"
reference = torch.load(sd_ckpt, map_location='cpu', weights_only=False)

# Mapping for renaming keys in reference
prefix_mapping = {
    "model.diffusion_model.": "model.",
    "cond_stage_model.model.": "text_stage_model.model.",
}

def replace_prefixes(state_dict, mapping):
    new_state_dict = {}
    for k, v in state_dict.items():
        new_key = k
        for old_prefix, new_prefix in mapping.items():
            if k.startswith(old_prefix):
                new_key = new_prefix + k[len(old_prefix):]
                break  # Only apply first match
        new_state_dict[new_key] = v
    return new_state_dict

# Process reference checkpoint
if 'state_dict' in reference:
    ref_state_dict = replace_prefixes(reference['state_dict'], prefix_mapping)
else:
    ref_state_dict = replace_prefixes(reference, prefix_mapping)

In [10]:

# ---------- New checkpoint (updated) ----------
train_ckpt = "/home/deanhazineh/Downloads/model_v5/model_snapshots/ckpt_last.ckpt"
updated = torch.load(train_ckpt, map_location='cpu')

# Extract new checkpoint state dict
if 'state_dict' in updated:
    new_state_dict = updated['state_dict']
else:
    new_state_dict = updated

In [11]:

# ---------- Compare and print new-only keys ----------
ref_keys = set(ref_state_dict.keys())
new_keys = set(new_state_dict.keys())

# Find keys in new checkpoint but not in reference
missing_keys = new_keys - ref_keys

print("\nParameters in the new checkpoint but missing in the reference checkpoint:")
for k in sorted(missing_keys):
    print(k)


Parameters in the new checkpoint but missing in the reference checkpoint:
cold_state
cond_stage_model.adapter.body.0.resnets.0.block1.bias
cond_stage_model.adapter.body.0.resnets.0.block1.weight
cond_stage_model.adapter.body.0.resnets.0.block2.bias
cond_stage_model.adapter.body.0.resnets.0.block2.weight
cond_stage_model.adapter.body.0.resnets.1.block1.bias
cond_stage_model.adapter.body.0.resnets.1.block1.weight
cond_stage_model.adapter.body.0.resnets.1.block2.bias
cond_stage_model.adapter.body.0.resnets.1.block2.weight
cond_stage_model.adapter.body.1.in_conv.bias
cond_stage_model.adapter.body.1.in_conv.weight
cond_stage_model.adapter.body.1.resnets.0.block1.bias
cond_stage_model.adapter.body.1.resnets.0.block1.weight
cond_stage_model.adapter.body.1.resnets.0.block2.bias
cond_stage_model.adapter.body.1.resnets.0.block2.weight
cond_stage_model.adapter.body.1.resnets.1.block1.bias
cond_stage_model.adapter.body.1.resnets.1.block1.weight
cond_stage_model.adapter.body.1.resnets.1.block2.bia

In [12]:
# Create a new state_dict with only the missing keys
missing_state_dict = {k: new_state_dict[k] for k in missing_keys}

# Optionally wrap it with a top-level 'state_dict' key
checkpoint_to_save = {
    'state_dict': missing_state_dict
}

# Save to a new checkpoint file
save_path = "/home/deanhazineh/Downloads/model_v5/missing_params_only.ckpt"
torch.save(checkpoint_to_save, save_path)

print(f"\nSaved {len(missing_keys)} parameters to {save_path}")



Saved 381 parameters to /home/deanhazineh/Downloads/model_v5/missing_params_only.ckpt
